## SFTTrainer Example - Finetuning with Sagemaker

This notebook demonstrates basic user flow for SFT Finetuning from a model available in Sagemaker Jumpstart.
Information on available models on jumpstart: https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-latest.html

### Setup and Configuration

Initialize the environment by importing necessary libraries and configuring AWS credentials

In [1]:
from sagemaker.train.sft_trainer import SFTTrainer
from sagemaker.train.common import TrainingType
from sagemaker.core.training.configs import InputData
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage

import boto3
from sagemaker.core.helper.session_helper import Session
import os


# For MLFlow native metrics in Trainer wait, run below line with approriate region
os.environ["SAGEMAKER_MLFLOW_CUSTOM_ENDPOINT"] = "https://mlflow.sagemaker.us-west-2.app.aws"



[12/03/25 12:48:28] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=804206;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=862910;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/jamjee/Library/Application Support/sagemaker/config.yaml


[12/03/25 12:48:29] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=28678;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=394871;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/botocore/credentials.py#1392\1392]8;;\

## Finetuning with Jumpstart base model

### Create SFTTrainer
**Required Parameters** 

* `model`: base_model id on Sagemaker Hubcontent that is available to finetune (or) ModelPackage artifacts

**Optional Parameters**
* `training_type`: Choose from TrainingType Enum(sagemaker.modules.train.common) either LORA OR FULL.
* `model_package_group_name`: ModelPackage group name or ModelPackageGroup
* `mlflow_resource_arn`: MLFlow app ARN to track the training job
* `mlflow_experiment_name`: MLFlow app experiment name(str)
* `mlflow_run_name`: MLFlow app run name(str)
* `training_dataset`: Training Dataset - either Dataset ARN or S3 Path of the dataset (Please note these are required for a training job to run, can be either provided via Trainer or .train())
* `validation_dataset`: Validation Dataset - either Dataset ARN or S3 Path of the dataset
* `s3_output_path`: S3 path for the trained model artifacts

In [4]:
# For fine-tuning 
sft_trainer = SFTTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage]
    training_type=TrainingType.LORA, 
    model_package_group_name="arn:aws:sagemaker:us-west-2:729646638167:model-package-group/sdk-test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/sft/", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    accept_eula=True
)


[12/03/25 12:48:48] INFO     SageMaker session not provided. Using default Session.                  ]8;id=501239;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=401765;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

In [3]:
# For fine-tuning (us-east-1)
sft_trainer = SFTTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage]
    training_type=TrainingType.LORA, 
    model_package_group_name="arn:aws:sagemaker:us-east-1:729646638167:model-package-group/sdk-test-finetuned-models-us-east-1", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-east-1:729646638167:mlflow-app/app-J2NPD6IV77BJ",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing-us-east-1/input_data/sft/train_285.jsonl", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing-us-east-1/output/",
    #sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/Admin"
)

[12/03/25 12:48:39] INFO     SageMaker session not provided. Using default Session.                  ]8;id=100876;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=949427;file:///Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # For fine-tuning (us-east-1)                                                               │
│ ❱  2 sft_trainer = SFTTrainer(                                                                   │
│    3 │   model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage]           │
│    4 │   training_type=TrainingType.LORA,                                                        │
│    5 │   model_package_group_name="arn:aws:sagemaker:us-east-1:729646638167:model-package-gro    │
│                                                                                                  │
│ /Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/ │
│ sft_trainer.py:143 in __init__                                                                   │
│                                                                                                  │
│   140 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │    ))                    │
│   141 │   │                                                                                      │
│   142 │   │   # Validate and set EULA acceptance                                                 │
│ ❱ 143 │   │   self.accept_eula = _validate_eula_for_gated_model(model, accept_eula, is_gated_m   │
│   144 │                                                                                          │
│   145 │   @_telemetry_emitter(feature=Feature.MODEL_CUSTOMIZATION, func_name="SFTTrainer.train   │
│   146 │   def train(self, training_dataset: Optional[Union[str, DataSet]] = None, validation_d   │
│                                                                                                  │
│ /Users/jamjee/workplace/sagemaker-python-sdk/venv3/lib/python3.12/site-packages/sagemaker/train/ │
│ common_utils/finetune_utils.py:693 in _validate_eula_for_gated_model                             │
│                                                                                                  │
│   690 │                                                                                          │
│   691 │   # Validate EULA acceptance for gated models                                            │
│   692 │   if is_gated_model and not accept_eula:                                                 │
│ ❱ 693 │   │   raise ValueError(                                                                  │
│   694 │   │   │   f"Model '{model}' is a gated model and requires EULA acceptance. "             │
│   695 │   │   │   "Please set accept_eula=True to proceed with training."                        │
│   696 │   │   )                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Model 'meta-textgeneration-llama-3-2-1b-instruct' is a gated model and requires EULA acceptance. Please
set accept_eula=True to proceed with training.

### Discover and update Finetuning options

Each of the technique and model has overridable hyperparameters that can be finetuned by the user.

In [3]:
print("Default Finetuning Options:")
pprint(sft_trainer.hyperparameters.to_dict()) # rename as hyperparameters

Default Finetuning Options:


{
│   'data_path': '',
│   'global_batch_size': '8',
│   'learning_rate': '0.0001',
│   'lr_warmup_ratio': '0.1',
│   'max_epochs': '5',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   'name': 'example-name-nh0f7',
│   'output_path': '',
│   'results_directory': '',
│   'resume_from_path': '',
│   'train_val_split_ratio': '0.9',
│   'training_data_name': '',
│   'validation_data_name': '',
│   'validation_data_path': ''
}

In [4]:
# To update any hyperparameter, simply assign the value, example:
sft_trainer.hyperparameters.global_batch_size=16

#### Start SFT training


In [ ]:
training_job = sft_trainer.train(
    wait=True,
)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-sft-20251203124855      │
│  MLFlow URL            ]8;id=851328;https://app-KPJZRTUVNUYS.mlflow.sagemaker.us-west-2.app.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IlVBSjJOQiIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNEZtUXhqZW9WN25wUTF6cm1CY1BkdlBVbFQ2OHJOQzl5QmljMjllSllXdGdBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVGdGFqSkJSekZ5VlROUWEwaENVbFpvYUhkd2F6WmtRbWd4V0ROdlNVUXJSV1kxYVUxR1QyMXdNMDFSTkhNNVpXcExLMHhLTkVsak5sazJReTlZUVVSWlp6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzVOell4T1RNeU5UUTFOemM2YTJWNUwyVmhNbUZrTUdSa0xXRmtNV0l0TkROalppMDRaak0xTFRNMU5UWmlZVE5pWm1NeE1nQzRBUUlCQUhqRHJwaCtQMVNKcHUzZUl2YnVXUi94bGQ2THMxNjhEU3JZbXo5akp0dlBUQUdGUFVjeUkxQXNycHM5bHJuTzF0RU9BQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU0raVd3ajZIVmlTSjJ0YWtFQWdFUWdEdkk0Y0lSU09ybnVWank0Rmcwd3JsVFU4RDkzNGxKRThnZ2VnQmNoR2JaYzE2UTZSZXZxdks3bGMzTGNmVEkzQjVoeUpjRFJmR2poemsvRmdJQUFCQUFZMldGWlkrUzNrM2w5dG0xRDkxN3ljS3A2WC9zTUZNRk9UQWtORmVnMUdJVjJYZUc5Snl2WVZiOGFqd3NzNVpTLy8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBVHpYWnJLQlQ5UXoyendwTjdkQnIzMHFqczl5anZYKzAvaTZ2U0RCTnpRYzNCQlErMDYzdUNiWHBIRnY1Y0VJcGlHSDNGMEZ5QWtUSWY5L24wSVdJalJIQ2cwODExZk5CWVFuQnhON3loZm0rSEdBV3NwK1pDNmNxbTJySEZRaDc4dGF6azJyaWlnWTB1aEdxTi9TakhBOWdCdjAvemZ3YnlIM1ZYdTlOelBIaVRmWnRCSUpiN0Vxbnk2ZkNyb3E3bXgyZWJYNkdjZUo0bFk4S0t1eTlaWW94dkVoQ1JCb0dJOEJMZ1h3NHU3dElCbGpsK1N0cVM2TVVRMHpzMGpLNk4vSm11TEh4U2JtejFxV1dqWTRJYkt4SDhpSkpxWk91MFI2TVBheVZ5UjZMZzNNbHE1MkJnY1ZCUE4wMW1hdmUrUjlmZ3FSLzRjSGhuYVgyTWIwSTNpL0FFa2dIRDlLRG14WURqZGpCdmNscEtGTjBFUzNQbzN5U1BiazNQTVlPMTY4SkFFcUFIRXVDOE1ka2h0YTF2Q3B3WEFwaUhZZmdIRXNJbWVhS0ZMSnNSV0RTdEZ0bDlaL3dRdTVXVk9IQXFxS3dGR0V6bStpK0MrSHlUeVZnZTM3M2Q4VXZtVTBEQVpHYWJ4Y0QvNzNPUXNINGZIVUJaVURTRlZHKzFieFVsV3Y4UWhpWTlZbm9SbjZZUmw3dHByWWhMOHorc2I3SUNXdUdsOXhxdzF3Q0JHNjRwYWFnbHJFWkNQVEtpRUpyeEl0SmJUS2dWNUZSc1daN2o1ZUJwajBZRWF3cndKOE1Kc0s0TEp2dFJWTnBBdERSdVkwQWk1Q2M4N2Rjd2tlSFpWdHVWbnd2T2p1Yk1DRnZLOU5nMUJXNmdLZmdKSlhkREErODBRbzVtUGZRbzR4eS8yUW5aQkQ4TjZKODZJbTMrQ00zVkNEL1hrdXFaQ1pXTkhmMGw4K3hFRWpQL2poZVdMQWtMcksyTkprY3RQWmMzek9uVlA0TUZaYjk0dzAyQWJaT3Z5VVo0RFViQUk4eDVKK2MrYUFzb0lFemkveWMzU3hrbnF2THg3TFZXSGdKallsSFczYWxQK2RiWVk1cUg3c0RkRHZTc3E5WENzOUVBMmZia3BqL0F1cjRhUjY3bWNNKzJWVVRIVmI1WHlBUG1IbmI3R1dCOVByZnJxc1JBS3l3aUxKY1Y3L0lBUVB3QTZHckEwT3FyYUJBcXk1dTRUc0NuR1JLZ1EwVjh4NDhKOFBQOUgwVGNYMU9INVNBR1E0NDVMVktvVDBtK1o0UWpEdHBjUC82UGd5YWR3ZkZBOWxJWWtkL29INUx5Sy84YlZQSzVrQnFId2JHUnFkYWcvdEpKbmJaYWh5RWs5VncwWVNpdWlENWp0eFgvTkdvV3ZLZzZrMU03SVUwN3hMSlcrY3lSVHJObG16T0ozRzBmQ3RSQ0paekswc0JINGJsbkVzQ1N0YVB3ZEhaZEV6bVFvUE1RMzM3eExCWWg4ZDc5NnA0TEk1QURYNVJzVkwxdkJsL3o0TEdxaFdybVRmajh0THJuVnlPTXo5L25pQmY2eVgycW12dDIvLzd5dEhqNk1xV0ZzYkZGa2lDU3psd0ozL2t4TWE0Mk9ZZ1FOSnh6ZTBKMWp4S1MvUGFYRjYwUENWc3hGVC9PM1luMXB3Vk5Yb2pHM0lCN0N3ckd4RnpiWkdvVTdnbDVCR0VWN0VJRDFWeU96YldUdnBpMUd3K0pEU2w3bm5aL0RsMDU3dVJMbmIyS3hpTXFFWHJFOGZXS252SGRSQnZqU2VqbEkxaDQxcElESC9jaWp1WE1nQU5JVm1tZDA4Umc5ZXNmbzQwbG8rRGxLbE1pNWo5MGhaVU1JWTV2SjBhNUoyeFNSekx0Zy82cUZYQjlxK1ZnY3ZVOG5aM1hIM3BaZ1IrU2VoYzBtZ0xKR2ZudGNRZG96MkZQOFl1VlZPQ0kzbkJHWVNoMStxRkc0SlVKMjdXdms0NEk0cDdJdVprSDYzdXNRQ1E1bERMenBFVjd0UkY0UWhyYlQ2bWx2bVJtaEtMNmUrSmo2eGltWFQxd1I1OStjZ3FWRGhtKzF1ZjBkaXdpWjgyWnhaMm5QcFRKRXFSeEpiUEp3VW1NTEpSMVlwWnBpNkdHeXA4Um1kNjloOElRY2VnZlNrYXp1T1B2Q0UrSjdTK2ZZdjhFMTB4TVllUm0xbEgvMmZjK0M3TkZBaWhNWlZDR0pKZFRBTDVmUmdMcmc3b0pQM0VJQVJubW5LQVhMV0FVQVp6QmxBakE4cGtrOE9zSHJERm1EeHlFMDh3R0o3R2lCLzFwNzkzSUx4ek5HM2VrNlg4S1pDTFh5cFV3MkhuVDRxQllLUmJrQ01RQ2loUlFONUZ0a0hDMlVwdlV1ZkF1NjZOQWFPOGp0RGVLMEpRZktiaWpnUWpnZU5PUTF0TTU0QVNCMWhieWs4Znc9IiwiY2lwaGVyVGV4dCI6IkFRSUJBSGpEcnBoK1AxU0pwdTNlSXZidVdSL3hsZDZMczE2OERTclltejlqSnR2UFRBSFZpV1pJK3pZOWdXV2U0Rk5uZzlJYUFBQUFvakNCbndZSktvWklodmNOQVFjR29JR1JNSUdPQWdFQU1JR0lCZ2txaGtpRzl3MEJCd0V3SGdZSllJWklBV1VEQkFFdU1CRUVETnVyZWdxa0R3REplY3dGWlFJQkVJQmI3UTN5bURlRnRma1p0bFpHeEJwRGp1QUNGYkprRHNTaHp

In [4]:
training_job = sft_trainer.train(
    wait=True,
)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921      │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          423.0s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             19.3s                     │
│   ✓    Pending           Preparing the instances for           11.8s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        15.6s                     │
│   ✓    Training          Training image download completed.    361.5s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    13.6s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:


from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921',
│   training_job_arn='arn:aws:sagemaker:us-west-2:729646638167:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-nh0f7',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   role_arn='arn:aws:iam::729646638167:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/input_data/sft/',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://mc-flows-sdk-testing/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   ),
│   creation_time=datetime.datetime(2025, 12, 1, 11, 49, 22, 717000, tzinfo=tzlocal()),
│   training_start_time=datetime.datetime(2025, 12, 1, 11, 49, 53, 819000, tzinfo=tzlocal()),
│   training_end_time=datetime.datetime(2025, 12, 1, 11, 5

In [ ]:
#In order to skip waiting and monitor the training Job later

'''
training_job = sft_trainer.train(
    wait=False,
)
'''

In [6]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251201172445',
│   training_job_arn='arn:aws:sagemaker:us-west-2:729646638167:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201172445',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201172445/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201172445/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-nh0f7',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   role_arn='arn:aws:iam::729646638167:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/input_data/sft/',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://mc-flows-sdk-testing/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11c8a9d00>
│   ),
│   creation_time=datetime.datetime(2025, 12, 1, 17, 24, 47, 482000, tzinfo=tzlocal()),
│   training_start_time=datetime.datetime(2025, 12, 1, 17, 25, 0, 593000, tzinfo=tzlocal()),
│   training_end_time=datetime.datetime(2025, 12, 1, 17, 31

### View any Training job details

We can get any training job details and its status with TrainingJob.get(...)

In [8]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lora_alpha': '32',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-4xpu2',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_schedules_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   training_plan_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_placement_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   max_p

## Continued Finetuning (or) Finetuning on Model Artifacts

#### Discover a ModelPackage and get its details

In [6]:
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage, ModelPackageGroup

#model_package_iter = ModelPackage.get_all(model_package_group_name="test-finetuned-models-gamma")
model_package = ModelPackage.get(model_package_name="arn:aws:sagemaker:us-west-2:729646638167:model-package/sdk-test-finetuned-models/2")

pprint(model_package)

ModelPackage(
│   model_package_name=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   model_package_group_name='sdk-test-finetuned-models',
│   model_package_version=2,
│   model_package_registration_type='Logged',
│   model_package_arn='arn:aws:sagemaker:us-west-2:729646638167:model-package/sdk-test-finetuned-models/2',
│   model_package_description=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   creation_time=datetime.datetime(2025, 12, 1, 11, 56, 23, 662000, tzinfo=tzlocal()),
│   inference_specification=InferenceSpecification(
│   │   containers=[
│   │   │   ContainersItem(
│   │   │   │   model_data_url=None,
│   │   │   │   image=None,
│   │   │   │   nearest_model_name=None,
│   │   │   │   model_data_source=ModelDataSource(
│   │   │   │   │   s3_data_source=S3ModelDataSource(
│   │   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201114921/output/model/',
│   │   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   │   compression_type='None',
│   │   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   │   manifest_s3_uri=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   │   e_tag=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   │   manifest_etag=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   │   │   │   )
│   │   │   │   ),
│   │   │   │   is_checkpoint=False,
│   │   │   │   base_model=BaseModel(
│   │   │   │   │   hub_content_name='meta-textgeneration-llama-3-2-1b-instruct',
│   │   │   │   │   hub_content_version='1.25.0',
│   │   │   │   │   recipe_name='llmft_llama3_2_1b_instruct_seq4k_gpu_sft_lora'
│   │   │   │   )
│   │   │   )
│   │   ]
│   ),
│   source_algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   validation_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   model_package_status='Completed',
│   model_package_status_details=ModelPackageStatusDetails(validation_statuses=[], image_scan_statuses=[]),
│   certify_for_marketplace=False,
│   model_approval_status=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   created_by=UserContext(
│   │   user_profile_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   user_profile_name=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   domain_id=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   iam_identity=IamIdentity(
│   │   │   arn='arn:aws:sts::729646638167:assumed-role/Admin/SageMaker',
│   │   │   principal_id='AROA2TYSUERLWFIUSFEJA:SageMaker',
│   │   │   source_identity=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   )
│   ),
│   metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   model_metrics=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   deployment_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   last_modified_time=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   last_modified_by=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   approval_description=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   domain=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   task=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   sample_payload_url=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   sample_payload_content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   customer_metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   drift_check_baselines=<sagemaker.core.utils.utils.Unassigned object at 0x11

#### Create Trainer

Trainer creation is same as above Finetuning Section except for `model`'s input is ModelPackage(previously trained artifacts)

In [7]:
# For fine-tuning 
sft_trainer = SFTTrainer(
    model=model_package, # Union[str, ModelPackage]
    training_type=TrainingType.LORA, 
    model_package_group_name="sdk-test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/sft/", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
)


[12/01/25 13:54:27] INFO     SageMaker session not provided. Using default Session.                  ]8;id=970170;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=204428;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

#### Start the Training

In [8]:
training_job = sft_trainer.train(
    wait=True,
)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-sft-20251201135434      │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          449.7s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             0.7s                      │
│   ✓    Pending           Preparing the instances for           13.1s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        66.0s                     │
│   ✓    Training          Training image download completed.    361.5s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    8.5s                      │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251201135434',
│   training_job_arn='arn:aws:sagemaker:us-west-2:729646638167:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201135434',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201135434/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251201135434/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-nh0f7',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   role_arn='arn:aws:iam::729646638167:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/input_data/sft/',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://mc-flows-sdk-testing/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1132e62a0>
│   ),
│   creation_time=datetime.datetime(2025, 12, 1, 13, 54, 35, 998000, tzinfo=tzlocal()),
│   training_start_time=datetime.datetime(2025, 12, 1, 13, 54, 49, 789000, tzinfo=tzlocal()),
│   training_end_time=datetime.datetime(2025, 12, 1, 14, 2

#### SFT Trainer Nova testing

In [15]:
os.environ['SAGEMAKER_REGION'] = 'us-east-1'

# For fine-tuning 
sft_trainer_nova = SFTTrainer(
    #model="test-nova-lite-v2", # Union[str, ModelPackage]
    #model="nova-textgeneration-micro",
    model="nova-textgeneration-lite-v2",
    training_type=TrainingType.LORA, 
    model_package_group_name="sdk-test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-east-1:052150106756:mlflow-app/app-UNBKLOAX64PX",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-nova-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-nova-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-east-1:729646638167:hub-content/sdktest/DataSet/sft-nova-test-dataset/0.0.1",
    s3_output_path="s3://mc-flows-sdk-testing-us-east-1/output/"
)


[12/02/25 13:21:09] INFO     SageMaker session not provided. Using default Session.                  ]8;id=278620;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=484609;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

In [16]:
sft_trainer_nova.hyperparameters.to_dict()

{'name': 'my-lora-sft-run-1r3op',
 'data_s3_path': 's3://my-bucket-name/train.jsonl',
 'output_s3_path': 's3://my-bucket-name/outputs/',
 'reasoning_enabled': 'True',
 'global_batch_size': '32',
 'max_steps': '10',
 'learning_rate': '1e-05',
 'lora_alpha': '64',
 'learning_rate_ratio': '64.0',
 'max_context_length': '8192'}

In [17]:
training_job = sft_trainer_nova.train(
    wait=True,
)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      nova-textgeneration-lite-v2-sft-20251202132123                    │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          4189.8s                                                           │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             20.4s                     │
│   ✓    Pending           Preparing the instances for           238.0s                    │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        320.2s                    │
│   ✓    Training          Training image download completed.    3004.3s                   │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    603.9s                    │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯